In [ ]:
import pickle

import numpy as np

new_pickle = 'data/dair-v2x-i/test.pkl'
with open('data/dair-v2x-i/dair_12hz_infos_val.pkl', 'rb') as f:
    data = pickle.load(f)
    
    x = data[0]

with open(new_pickle, 'wb') as f:
    pickle.dump([x], f)

print([x])

In [ ]:
import torch
ckpt_path = './ckpt/BEVHeight_R101_256_140.8_65.93_49_epochs.ckpt'
a = torch.load(ckpt_path)

print(a)

In [1]:
from argparse import ArgumentParser, Namespace

import os
import mmcv
import pytorch_lightning as pl
import cv2
import json
import torch
import torch.nn.parallel
import torch.utils.data
import torch.utils.data.distributed
import torchvision.models as models
from pytorch_lightning.core import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim.lr_scheduler import MultiStepLR

from dataset.nusc_mv_det_dataset import NuscMVDetDataset, collate_fn
from evaluators.det_evaluators import RoadSideEvaluator
from models.bev_height import BEVHeight
from utils.torch_dist import all_gather_object, get_rank, synchronize
from utils.backup_files import backup_codebase
from evaluators.result2kitti import *
from scripts.data_converter.visual_utils import *
from tqdm import tqdm
from exps.dair_v2x.bev_height_lss_r101_864_1536_256x256_140 import BEVHeightLightningModel

In [2]:
ckpt_path = './ckpt'
ckpt_name = os.listdir(ckpt_path)[0]
model_pth = os.path.join(ckpt_path, ckpt_name)
BEVHeight = BEVHeightLightningModel(batch_size_per_device=1).load_from_checkpoint(model_pth)

data_root = "data/dair-v2x-i/"
gt_label_path = "data/dair-v2x-i-kitti/training/label_2"

dataloader = BEVHeight.val_dataloader()
(sweep_imgs, mats, _, img_metas, _, _) = next(iter(dataloader))

with torch.no_grad():
    for key, value in mats.items():
        mats[key] = value.cuda()
    sweep_imgs = sweep_imgs.cuda()
    BEVHeight.model.cuda()
    preds = BEVHeight(sweep_imgs, mats)

    results = BEVHeight.model.get_bboxes(preds, img_metas)
    for i in range(len(results)):
        results[i][0] = results[i][0].tensor.detach().cpu().numpy()
        results[i][1] = results[i][1].detach().cpu().numpy()
        results[i][2] = results[i][2].detach().cpu().numpy()
        results[i].append(img_metas[i])
    # print(results[0])

    all_pred_results = list()
    all_img_metas = list()
    for validation_step_output in results:
        all_pred_results.append(validation_step_output[:3])
        all_img_metas.append(validation_step_output[3])
    synchronize()
    len_dataset = len(BEVHeight.val_dataloader().dataset)
    all_pred_results = sum(
        map(list, zip(*all_gather_object(all_pred_results))),     # !!!
        [])[:len_dataset]
    all_img_metas = sum(map(list, zip(*all_gather_object(all_img_metas))),
                        [])[:len_dataset]
    # print(all_img_metas)

    result_files, tmp_dir = BEVHeight.evaluator.format_results(all_pred_results, all_img_metas,
                                                               result_names=['img_bbox'],
                                                               jsonfile_prefix=None)     # !!!
    print('=' * 20)
    print(result_files, tmp_dir)
    print('=' * 20)
    results_path = "outputs"

    results_file = result_files["img_bbox"]
    dair_root = data_root
    demo = True

    category_map_dair = {"car": "Car", "van": "Car", "truck": "Car", "bus": "Car", "pedestrian": "Pedestrian",
                         "bicycle": "Cyclist", "trailer": "Cyclist", "motorcycle": "Cyclist"}

    with open(results_file, 'r', encoding='utf8') as fp:
        results = json.load(fp)["results"]
    for sample_token in tqdm(results.keys()):
        sample_id = int(sample_token.split("/")[1].split(".")[0])
        virtuallidar_to_camera_file = os.path.join(dair_root, "calib/virtuallidar_to_camera",    # Input!
                                                   "{:06d}".format(sample_id) + ".json")
        Tr_velo_to_cam, r_velo2cam, t_velo2cam = get_lidar2cam(virtuallidar_to_camera_file)
        preds = results[sample_token]
        pred_lines = []
        bboxes = []
        for pred in preds:
            loc = pred["translation"]
            dim = pred["size"]
            yaw_lidar = pred["box_yaw"]
            detection_score = pred["detection_score"]
            class_name = pred["detection_name"]

            w, l, h = dim[0], dim[1], dim[2]
            x, y, z = loc[0], loc[1], loc[2]
            bottom_center = [x, y, z]
            obj_size = [l, w, h]
            bottom_center_in_cam = r_velo2cam * np.matrix(bottom_center).T + t_velo2cam
            _, yaw = get_camera_3d_8points(
                obj_size, yaw_lidar, bottom_center, bottom_center_in_cam, r_velo2cam, t_velo2cam
            )
            yaw = 0.5 * np.pi - yaw_lidar

            cam_x, cam_y, cam_z = convert_point(np.array([x, y, z, 1]).T, Tr_velo_to_cam)
            box = get_lidar_3d_8points([w, l, h], yaw_lidar, [x, y, z + h / 2])
            if detection_score > 0.45 and class_name in category_map_dair.keys():
                i1 = category_map_dair[class_name]
                i9, i11, i10 = str(round(h, 4)), str(round(w, 4)), str(round(l, 4))
                i12, i13, i14 = str(round(cam_x, 4)), str(round(cam_y, 4)), str(round(cam_z, 4))
                i15 = str(round(yaw, 4))
                line = [i1, '0', '0', '0', '0', '0', '0', '0', i9, i10, i11, i12, i13, i14, i15,
                        str(round(detection_score, 4))]
                pred_lines.append(line)
                bboxes.append(box)
        os.makedirs(os.path.join(results_path, "data"), exist_ok=True)
        write_kitti_in_txt(pred_lines, os.path.join(results_path, "data", "{:06d}".format(sample_id) + ".txt"))
        if demo:
            os.makedirs(os.path.join(results_path, "demo"), exist_ok=True)
            label_path = os.path.join(gt_label_path, "{:06d}".format(sample_id) + ".txt")
            demo_file = os.path.join(results_path, "demo", "{:06d}".format(sample_id) + ".jpg")
            pcd_vis(bboxes, demo_file, label_path, Tr_velo_to_cam)

2024-04-29 13:20:32,846 - mmcv - INFO - initialize SECONDFPN with init_cfg [{'type': 'Kaiming', 'layer': 'ConvTranspose2d'}, {'type': 'Constant', 'layer': 'NaiveSyncBatchNorm2d', 'val': 1.0}]
2024-04-29 13:20:32,859 - mmcv - INFO - 
deblocks.0.0.weight - torch.Size([128, 256, 4, 4]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2024-04-29 13:20:32,860 - mmcv - INFO - 
deblocks.0.1.weight - torch.Size([128]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2024-04-29 13:20:32,860 - mmcv - INFO - 
deblocks.0.1.bias - torch.Size([128]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2024-04-29 13:20:32,861 - mmcv - INFO - 
deblocks.1.0.weight - torch.Size([128, 512, 2, 2]): 
The value is the same before and after calling `init_weights` of SECONDFPN  
 
2024-04-29 13:20:32,861 - mmcv - INFO - 
deblocks.1.1.weight - torch.Size([128]): 
The value is the same before and after calling `init_weig


Formating bboxes of img_bbox
Start to convert detection format...
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2/2, 166.5 task/s, elapsed: 0s, ETA:     0s
Results writes to ./outputs/bev_height_lss_r101_864_1536_256x256/results_nusc.json
{'img_bbox': './outputs/bev_height_lss_r101_864_1536_256x256/results_nusc.json'} <TemporaryDirectory '/tmp/tmpk2b3n1v7'>


100%|██████████| 2/2 [00:00<00:00, 17.76it/s]


In [7]:
from PIL import Image

sweep_img = sweep_imgs[0, 0, 0, ...]
print(sweep_img.shape)
print(sweep_img)
img = sweep_img.squeeze(0).permute(1, 2, 0).cpu().numpy()

# Image.fromarray(img.astype(np.uint8)).show()

torch.Size([3, 864, 1536])
tensor([[[-1.3302, -1.3130, -1.2788,  ..., -1.5699, -1.5357, -1.4500],
         [-1.3302, -1.3130, -1.2788,  ..., -1.5699, -1.5357, -1.4500],
         [-1.3302, -1.3130, -1.2788,  ..., -1.5528, -1.5185, -1.4500],
         ...,
         [-0.6794, -0.6794, -0.6794,  ..., -1.3302, -1.3302, -1.3302],
         [-0.6623, -0.6623, -0.6623,  ..., -1.3302, -1.3302, -1.3302],
         [-0.6623, -0.6623, -0.6623,  ..., -1.3302, -1.3302, -1.3302]],

        [[-1.2129, -1.1954, -1.1604,  ..., -1.3354, -1.2479, -1.1779],
         [-1.2129, -1.1954, -1.1604,  ..., -1.3354, -1.2479, -1.1779],
         [-1.2129, -1.1954, -1.1604,  ..., -1.3529, -1.2654, -1.1779],
         ...,
         [-0.6176, -0.6176, -0.6176,  ..., -1.3179, -1.3179, -1.3179],
         [-0.6001, -0.6001, -0.6001,  ..., -1.3179, -1.3179, -1.3179],
         [-0.6001, -0.6001, -0.6001,  ..., -1.3179, -1.3179, -1.3179]],

        [[-0.8110, -0.7936, -0.7587,  ..., -1.1944, -1.1421, -1.0550],
         [-0.8110,

In [16]:
from PIL import Image
from dataset.nusc_mv_det_dataset import NuscMVDetDataset, img_transform
H = 1080
W = 1920
final_dim = (864, 1536)

img_conf = dict(img_mean=[123.675, 116.28, 103.53],
                img_std=[58.395, 57.12, 57.375],
                to_rgb=True)

data_root = "data/dair-v2x-i/"
gt_label_path = "data/dair-v2x-i-kitti/training/label_2"

img_path = os.path.join('data/dair-v2x-i/image', '000018.jpg')
img1 = Image.open(img_path)

resize = 0.8
resize_dims = (1536, 864)
crop = (0, 0, 1536, 864)
flip = False
rotate_ida = 0
img_mean = np.array(img_conf['img_mean'])
img_std = np.array(img_conf['img_std'])
to_rgb = True

img1, ida_mat = img_transform(
                    img1,
                    resize=resize,
                    resize_dims=resize_dims,
                    crop=crop,
                    flip=flip,
                    rotate=rotate_ida,
                )
img1 = mmcv.imnormalize(np.array(img1), img_mean, img_std, to_rgb)
img1 = torch.from_numpy(img1).permute(2, 0, 1)
print(img1.shape)
print(sweep_imgs.shape)
# print(img1)

torch.Size([3, 864, 1536])
torch.Size([2, 1, 1, 3, 864, 1536])


In [ ]:
mat = {k: v[0:1, ...] for k, v in mats.items()}
print(mats['sensor2ego_mats'].shape)
print(mat['sensor2ego_mats'].shape)

for k,v in mat.items():
    print(f"{k}: {v}\n")

In [ ]:
mat = {k: v[1:2, ...] for k, v in mats.items()}
# ida denotes image data augmentation
# bda denotes bev data augmentation
for k,v in mat.items():
    print(f"{k}: {v}\n")

In [ ]:
sweep_img = sweep_imgs[0:1, ...]
print(sweep_imgs.shape)
print(sweep_img.shape)

In [ ]:
print(img_metas[0])
print(img_metas[1])

In [ ]:
data_root = "data/dair-v2x-i/"

virtuallidar_to_camera_file = os.path.join(dair_root, "calib/virtuallidar_to_camera",    # Input!
                                                       "{:06d}".format(sample_id) + ".json")
Tr_velo_to_cam, r_velo2cam, t_velo2cam = get_lidar2cam(virtuallidar_to_camera_file)

In [ ]:
print(Tr_velo_to_cam)

In [ ]:
print(r_velo2cam)

In [ ]:
print(t_velo2cam)

In [30]:
import json
def format_json(json_file, is_print: bool = False):
     return json.dumps(json_file, sort_keys=True, indent=4, separators=(', ', ': '), ensure_ascii=False)

with open('outputs/bev_height_lss_r101_864_1536_256x256/results_nusc1.json', 'r+') as f:
    results_nusc = json.load(f)
    f.truncate()
    f.write(format_json(results_nusc))